# Charsiu aligner

## Libs

In [2]:
import os
import sys
from ssd_paths import *
import pandas as pd
from misc_progress_bar import draw_progress_bar

# change this path to where you saved the charsiu package
charsiu_dir = os.path.join(ssd_aishell_path, 'charsiu')
os.chdir(charsiu_dir)

sys.path.append('%s/src/' % charsiu_dir)
from Charsiu import charsiu_forced_aligner

NameError: name 'ssd_aishell_path' is not defined

In [35]:
charsiu = charsiu_forced_aligner(aligner='charsiu/zh_xlsr_fc_10ms', lang='zh')

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of the model checkpoint at charsiu/zh_xlsr_fc_10ms were not used when initializing Wav2Vec2ForFrameClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForFrameClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForFrameClassification were not initialized from the model checkpoint at charsiu/zh_xlsr_fc_10ms and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.

## Trial

In [31]:
wave_file = os.path.join(as_wav_path, "SSB00050017.wav")
sentence_parts = "他 ta1 前 qian2 天 tian1 举 ju3 办 ban4 粉 fen3 丝 si1 见 jian4 面 mian4 会 hui4".split()
chinese_chars = ''.join(sentence_parts[0::2])
pinyin = ' '.join(sentence_parts[1::2])

In [37]:
res = charsiu.align(audio=wave_file, text=chinese_chars)
print(res)

In [38]:
# charsiu.serve(audio = wave_file, text = chinese_chars, save_to = ssd_aishell_path + 'test.TextGrid')

Alignment output has been saved to /media/ldlmdl/A2AAE4B1AAE482E1/SSD_Documents/featln/src/aishell/test.TextGrid


## Process

### Get by-speaker lists

In [36]:
# Get the list of filenames in the directory
filenames = sorted(os.listdir(as_wav_path))

# Create a dictionary to store the speakers
speakers = {}

# Categorize the filenames into speakers
for filename in filenames:
    speaker = filename[:7]
    if speaker not in speakers:
        speakers[speaker] = []
    speakers[speaker].append(filename)

In [37]:
kotoba = pd.read_csv(os.path.join(ssd_aishell_path, 'aishell_transcript.csv'))
kotoba

,rec,chinese,pinyin
0,SSB00050001.wav,广州女大学生登山失联四天警方找到疑似女尸,guang3 zhou1 nv3 da4 xue2 sheng1 deng1 shan1 s...
1,SSB00050002.wav,尊重科学规律的要求,zhun1 zhong4 ke1 xue2 gui1 lv4 de5 yao1 qiu2
2,SSB00050003.wav,七路无人售票,qi1 lu4 wu2 ren2 shou4 piao4
3,SSB00050004.wav,黑客宣布只要拨打某一个电话,hei1 ke4 xuan1 bu4 zhi3 yao4 bo1 da2 mou3 yi2 ...
4,SSB00050005.wav,北京万科总经理刘肖的观点极具代表性,bei3 jing1 wan4 ke1 zhong3 jing1 li3 liu2 xiao...
...,...,...,...
63257,SSB19560477.wav,情歌对唱,qing2 ge1 dui4 chang4
63258,SSB19560478.wav,大家是不是会更惊喜一些呢,da4 jia1 shi4 bu2 shi4 hui4 geng4 jing1 xi3 yi...
63259,SSB19560479.wav,患抑郁症老汉十几年足不出户身背五四次违章记录,huan4 yi4 yu4 zheng4 lao3 han4 shi2 ji3 nian2 ...
63260,SSB19560480.wav,一百三十五万九千一百二十三,yi4 bai3 san1 shi2 wu3 wan4 jiu3 qian1 yi1 bai...


In [38]:
for speaker, rec_list in speakers.items(): 
    df = pd.DataFrame(columns=['speaker', 'rec', 'idx', 'start_time', 'end_time', 'token', 'duration'])
    total = len(rec_list)
    for idx, rec in enumerate(sorted(rec_list)): 
        if rec.endswith('.wav'):
            draw_progress_bar(idx, total)
            wave_file = os.path.join(as_wav_path, rec)
            chinese_chars = kotoba[kotoba["rec"] == rec]["chinese"].item()
            res = charsiu.align(audio=wave_file, text=chinese_chars)
            starttimes, endtimes, tokens = list(zip(*res[0]))
            add_content = pd.DataFrame({'speaker':speaker, 'rec':rec.split(".")[0], 'idx':list(range(len(tokens))), 'start_time': starttimes, 'end_time': endtimes, 'token': tokens})
            add_content['duration'] = add_content['end_time'] - add_content['start_time']
            df = pd.concat([df, add_content], ignore_index=True)
    df.to_csv(os.path.join(ssd_aishell_path, 'phones_extract', speaker + ".csv"), index=False)

[                                                  ] 0%	

/tmp/ipykernel_115078/4030351444.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, add_content], ignore_index=True)


[                                                  ] 0%		

/tmp/ipykernel_115078/4030351444.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, add_content], ignore_index=True)


[                                                  ] 0%		

...

/tmp/ipykernel_115078/4030351444.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, add_content], ignore_index=True)


[                                                  ] 0%		

/tmp/ipykernel_115078/4030351444.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, add_content], ignore_index=True)


[====                                              ] 8%	

KeyboardInterrupt: 

## For use of testing only. Therefore we don;t need to generate all segments. 

## Combine into log

In [3]:
import os
import sys
from ssd_paths import *
import pandas as pd

In [4]:
def csv_bind(log_dir): 
    # List all the CSV files in the directory that start with 's'
    directory = log_dir
    csv_files = sorted([f for f in os.listdir(directory) if f.startswith('S') and f.endswith('.csv')])

    # Read and concatenate the CSV files using pandas
    dfs = []
    for file in csv_files:
        df = pd.read_csv(os.path.join(directory, file))
        dfs.append(df)

    concatenated_df = pd.concat(dfs, ignore_index=True)

    # Save the concatenated dataframe as "log.csv"
    concatenated_df.to_csv(os.path.join(directory, 'log.csv'), index=False)

In [ ]:
csv_bind(as_phones_extract_path)